<a href="https://colab.research.google.com/github/Pavun-KumarCH/Skin-cancer-detection-VGG16-Model/blob/main/Skin-Cancer-Detection-CNN-VGG16-Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Skin Cancer Classification with Machine Learning
So this is where a machine learning algorithm works in the classification of Skin Cancer. As I mentioned earlier Skin Cancer can be easily cured in the early stages of the disease, but it’s the people who don’t want to visit the doctor.

So here is a simple machine learning algorithm, which can help those people to identify if they are having skin cancer or not while sitting at their homes. This Machine Learning algorithm is based on Convolutional Neural Networks (CNN).

# CNN Layer Classification for skin cancer detection


In [ ]:
# Let Import required libraries
import time
import numpy as np
from skimage import io
import matplotlib.pyplot as plt
import tensorflow as tf
import tf_keras as keras

In [ ]:
# import required tensorfloe modules
from keras.layers import InputLayer, Conv2D, MaxPool2D, Dense, Flatten, Dropout

Now I will simply upload images to train our machine learning model using the skimage library in python.

In [ ]:
imgb = io.imread('b1.png')
imgm = io.imread('m1.png')

These images are sample images of Benign mole and Malign mole which are a type of skin problems.

Lets visualize this images

In [ ]:
plt.figure(figsize = (10, 20))
plt.subplot(1, 2, 1)
plt.imshow(imgb)
plt.title("Benign")
plt.axis('off')
plt.subplot(1, 2, 2)
plt.imshow(imgm)
plt.title('Malign')
plt.axis('off')

# Lets format the images to our requires size

In [ ]:
def format_image(image):
  image = tf.image.resize(image[:, : ,:3],(128, 128))/255.
  image = np.expand_dims(image, axis = 0)
  return image

imgb1 = format_image(imgb)

print(imgb.shape)
print(imgb1.shape)

#Lets creates a model with the VGG16 architecture followed by a sequential model for binary classification.

In [ ]:
# VGG16 architecture

model  = tf.keras.Sequential([

    # Input layer
    tf.keras.layers.InputLayer(input_shape = (128, 128, 3), name = 'Input_Layer'),

    # Block 1
    Conv2D(64, (3,3), padding = 'same', activation = 'relu', name = 'Block1_conv1'),
    Conv2D(64, (3,3), padding = 'same', activation = 'relu', name = 'Block1_Conv2'),
    MaxPool2D((2,2), strides = (2,2), name = 'Block__Maxpool'),

    # Block 2
    Conv2D(128, (3,3), padding = 'same', activation = 'relu', name = 'Block2_conv1'),
    Conv2D(128, (3,3), padding = 'same', activation = 'relu', name = 'Block2_conv2'),
    MaxPool2D((2,2), strides = (2,2), name = 'Block2_Maxpool'),

    # Block 3
    Conv2D(256, (3,3), padding = 'same', activation = 'relu', name = 'Block3_conv1'),
    Conv2D(256, (3,3), padding = 'same', activation = 'relu', name = 'Block3_conv2'),
    Conv2D(256, (3,3), padding = 'same', activation = 'relu', name = 'Block3_conv3'),
    MaxPool2D((2,2), strides = (2,2), name = 'Block3_MaxPool'),

    # Block 4
    Conv2D(512, (3,3), padding = 'same', activation = 'relu', name = 'Block4_conv1'),
    Conv2D(512, (3,3), padding = 'same', activation = 'relu', name = 'Block4_conv2'),
    Conv2D(512, (3,3), padding = 'same', activation = 'relu', name = 'Block4_conv3'),
    MaxPool2D((2,2), strides = (2,2), name = 'Block4_Maxpool'),

    # Block 5
    Conv2D(512, (3,3), padding = 'same', activation = 'relu', name = 'Block5_conv1'),
    Conv2D(512, (3,3), padding = 'same', activation = 'relu', name = 'Block5_conv2'),
    Conv2D(512, (3,3), padding = 'same', activation = 'relu', name = 'Block5_conv3'),
    MaxPool2D((2,2), strides = (2,2), name = 'Block5_Maxpool'),

    # Flatten
    Flatten(),
    Dense(256, activation = 'relu'),
    Dense(units = 1, activation = 'sigmoid')
    ])

model.compile(optimizer = 'adam',
              loss = tf.keras.losses.BinaryCrossentropy(),
              metrics = ['accuracy'])

model.summary()

Lets save the model with time stamp



In [ ]:
t = time.time()
t
export_path = "./{}.h5".format(int(t))
model.save(export_path)

In [ ]:
# Saving the Model with weights and assets
tf.saved_model.save(model, 'VGG16 Skin-cancer-detection')

In [ ]:
layer_dict = dict([(layer.name, layer) for layer in model.layers])
layer_dict


##Now lets visualize the output of our trained model


This function is designed to extract the activations of a specific layer in a given neural network model when provided with an input image.

In [ ]:

from keras import backend as K

def active_viewer(model, layer_name, im_put):
    layer_dict = dict([(layer.name, layer) for layer in model.layers])
    layer = layer_dict[layer_name]
    # Create a function using tf.keras.backend.function
    activ_func = tf.keras.backend.function([model.input], [layer.output])
    # Compute activations using the created function
    activations = activ_func(im_put)
    return activations

active_benign = active_viewer(model,'Block2_conv1',imgb1)
active_benign


This function effectively reverses the preprocessing steps applied to neural network outputs, making them readable and suitable for visualization purposes.

In [ ]:
def deprocess_image(x):
  # normalize tensor: center on 0., ensure std is 0.1
  x -= x.mean()
  x /= (x.std() + 1e-5)
  x *= 0.1

  # clip [0, 1]
  x += 0.5
  x = np.clip(x, 0, 1)

  # Convert to RGB array
  x *= 255
  if K.image_data_format() == 'channels_first':
    x = x.transpose(1, 2, 0)
  x = np.clip(x, 0, 255).astype('uint8')
  return x

img_benign = deprocess_image(active_benign[0])
print(img_benign[0])

This function is designed to visualize filters of a convolutional neural network.

In [ ]:
def plot_filters(filters):
    new_image = np.zeros((16 * filters.shape[0], 8 * filters.shape[1]))  # Corrected shape initialization
    for i in range(filters.shape[2]):
        y = i % 8
        x = i // 8
        new_image[x * filters.shape[0]:x * filters.shape[0] + filters.shape[0],
                  y * filters.shape[1]:y * filters.shape[1] + filters.shape[1]] = filters[:, :, i]  # Corrected typo
    plt.figure(figsize=(10, 20))  # Moved outside the loop
    plt.imshow(new_image)
    plt.axis('off')
    plt.show()

plot_filters(img_benign[0])


Alternative

In [ ]:
for n in range(128):
  plt.subplot(8, 16, n+1)
  plt.imshow(img_benign[0,:,:,n])
  plt.axis('off')

Lets look at Malign

In [ ]:
activ_malign = active_viewer(model,'Block2_conv1',imgm.reshape(1,128,128,3))
img_malign = deprocess_image(activ_malign[0])
plot_filters(img_malign[0])
plt.figure(figsize=(20,20))


Now lets zoom and visualize some of the filters

In [ ]:
plt.figure(figsize=(10,10))
plt.subplot(121)
plt.imshow(img_benign[0,:,:,34])
plt.axis('off')
plt.subplot(122)
plt.imshow(img_malign[0,:,:,34])
plt.axis('off')

In [ ]:
plt.figure(figsize=(10,10))
plt.subplot(121)
plt.imshow(img_benign[0,:,:,80])
plt.axis('off')
plt.subplot(122)
plt.imshow(img_malign[0,:,:,80])
plt.axis('off')

Testing the Model
Now let’s visualize and test the actual ability of our trained model

In [ ]:
def plot_filters32(filters):
    newimage = np.zeros((16*filters.shape[0],16*filters.shape[1]))
    for i in range(filters.shape[2]):
        y = i%16
        x = i//16
        newimage[x*filters.shape[0]:x*filters.shape[0]+filters.shape[0],
                 y*filters.shape[1]:y*filters.shape[1]+filters.shape[1]] = filters[:,:,i]
    plt.figure(figsize = (15,25))
    plt.imshow(newimage)
    plt.axis('off')

active_benign = active_viewer(model,'Block3_conv1',imgb1.reshape(1,128,128,3))
img_benign = deprocess_image(active_benign[0])
plot_filters32(img_benign[0])

For Malign

In [ ]:
active_malign = active_viewer(model,'Block3_conv1',imgm.reshape(1,128,128,3))
img_malign = deprocess_image(active_malign[0])
plot_filters32(img_malign[0])